##### torch nn from scratch

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(2, 4)  # Input layer (2 neurons) -> Hidden layer (4 neurons)
        self.fc2 = nn.Linear(4, 1)  # Hidden layer (4 neurons) -> Output layer (1 neuron)

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Activation function
        x = self.fc2(x)
        return x

# Instantiate model, loss function, and optimizer
model = SimpleNN()
criterion = nn.MSELoss()  # Mean Squared Error loss for regression
optimizer = optim.SGD(model.parameters(), lr=0.05)  # Stochastic Gradient Descent


In [10]:
# Generate dummy input data (batch_size=3, input_size=2)
inputs = torch.tensor([[0.1, 0.2], [0.4, 0.5], [0.7, 0.8]], dtype=torch.float32)
targets = torch.tensor([[0.3], [0.6], [0.9]], dtype=torch.float32)  # Expected outputs

# Training loop for 5 epochs
for epoch in range(5):
    optimizer.zero_grad()  # Clear previous gradients
    
    outputs = model(inputs)  # Forward pass (PyTorch tracks gradients)
    
    loss = criterion(outputs, targets)  # Compute loss
    
    loss.backward()  # Backpropagation (Computes gradients)
    
    optimizer.step()  # Update model weights
    
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


Epoch 1, Loss: 0.04299898073077202
Epoch 2, Loss: 0.04028601571917534
Epoch 3, Loss: 0.03821534290909767
Epoch 4, Loss: 0.03660694137215614
Epoch 5, Loss: 0.03533036634325981


##### encoder, decoder inference with torch

In [12]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load Pre-trained Model and Tokenizer
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"  # Pre-trained sentiment analysis BERT
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Function to classify text
def classify_text(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    
    # Forward pass through BERT model
    with torch.no_grad():
        outputs = model(**inputs)

    # Get predicted class (highest probability)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    
    return predicted_class  # Returns class index (e.g., 0 for negative, 4 for very positive)

# Example usage
text = "This movie was absolutely fantastic!"
result = classify_text(text)
print(f"Predicted Sentiment Class: {result}")


c:\Users\Nishad_Khade\OneDrive - Dell Technologies\Desktop\UI_POC\env\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Nishad_Khade\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Predicted Sentiment Class: 4


In [17]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load Pre-trained GPT-2 Model and Tokenizer
model_name = "gpt2"  # Base GPT-2 model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Function for text completion
def generate_text(prompt, max_length=500):
    # Tokenize input text
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate text using GPT-2
    outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1, temperature=0.7)

    # Decode generated tokens back into text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Example usage
prompt = "The future of artificial intelligence is good."
result = generate_text(prompt)
print("Generated Text:\n", result)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:
 The future of artificial intelligence is good. It's not going to be a problem for the next 10 years. It's going to be a problem for the next 20 years. And it's going to be a problem for the next 20 years. And it's going to be a problem for the next 20 years. And it's going to be a problem for the next 20 years. And it's going to be a problem for the next 20 years. And it's going to be a problem for the next 20 years. And it's going to be a problem for the next 20 years. And it's going to be a problem for the next 20 years. And it's going to be a problem for the next 20 years. And it's going to be a problem for the next 20 years. And it's going to be a problem for the next 20 years. And it's going to be a problem for the next 20 years. And it's going to be a problem for the next 20 years. And it's going to be a problem for the next 20 years. And it's going to be a problem for the next 20 years. And it's going to be a problem for the next 20 years. And it's going to be a

##### placeholder

In [23]:
text = "The future of AI is"

# Tokenize
inputs = tokenizer(text, return_tensors="pt")

In [24]:
inputs

{'input_ids': tensor([[ 464, 2003,  286, 9552,  318]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [44]:
import inspect
def tool(func):
    """
    A decorator that creates a Tool instance from the given function.
    """
    # Get the function signature
    signature = inspect.signature(func)
    
    # Extract (param_name, param_annotation) pairs for inputs
    arguments = []
    for param in signature.parameters.values():
        annotation_name = (
            param.annotation.__name__ 
            if hasattr(param.annotation, '__name__') 
            else str(param.annotation)
        )
        arguments.append((param.name, annotation_name))
    
    # Determine the return annotation
    return_annotation = signature.return_annotation
    if return_annotation is inspect._empty:
        outputs = "No return annotation"
    else:
        outputs = (
            return_annotation.__name__ 
            if hasattr(return_annotation, '__name__') 
            else str(return_annotation)
        )
    
    # Use the function's docstring as the description (default if None)
    description = func.__doc__ or "No description provided."
    
    # The function name becomes the Tool name
    name = func.__name__
    
    # Return a new Tool instance
    return Tool(
        name=name, 
        description=description, 
        func=func, 
        arguments=arguments, 
        outputs=outputs
    )

In [53]:
@tool
def calculator(a: int, b: int) -> int:
    """Multiply two integers."""
    return a * b

print(calculator.to_string())

Tool Name: calculator, Description: Multiply two integers., Arguments: a: int, b: int, Outputs: int


In [55]:
import os
from huggingface_hub import InferenceClient

# os.environ["HF_TOKEN"]="hf_xxxxxxxxxxx"

client = InferenceClient("meta-llama/Llama-3.2-3B-Instruct")
# if the outputs for next cells are wrong, the free model may be overloaded. You can also use this public endpoint that contains Llama-3.2-3B-Instruct
client = InferenceClient("https://jc26mwg228mkj8dw.us-east-1.aws.endpoints.huggingface.cloud")

In [56]:
# As seen in the LLM section, if we just do decoding, **the model will only stop when it predicts an EOS token**, 
# and this does not happen here because this is a conversational (chat) model and we didn't apply the chat template it expects.
output = client.text_generation(
    "The capital of france is",
    max_new_tokens=100,
)

print(output)

 Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris.


In [57]:
# If we now add the special tokens related to Llama3.2 model, the behaviour changes and is now the expected one.
prompt="""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

The capital of france is<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
output = client.text_generation(
    prompt,
    max_new_tokens=100,
)

print(output)

...Paris!


In [58]:
output = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "The capital of france is"},
    ],
    stream=False,
    max_tokens=1024,
)

print(output.choices[0].message.content)

Paris.


# Lesson 2: Comparing Trained LLM Tokenizers

In this notebook of lesson 2, you will work with several tokenizers associated with different LLMs and explore how each tokenizer approaches tokenization differently. 

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Tokenizing Text

In this section, you will tokenize the sentence "Hello World!" using the tokenizer of the [`bert-base-cased` model](https://huggingface.co/google-bert/bert-base-cased). 

Let's import the `Autotokenizer` class, define the sentence to tokenize, and instantiate the tokenizer.

In [1]:
from transformers import AutoTokenizer

c:\Users\Nishad_Khade\OneDrive - Dell Technologies\Desktop\UI_POC\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# define the sentence to tokenize
sentence = "Hello world!"

In [3]:
# load the pretrained tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

c:\Users\Nishad_Khade\OneDrive - Dell Technologies\Desktop\UI_POC\env\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Nishad_Khade\.cache\huggingface\hub\models--bert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


You'll now apply the tokenizer to the sentence. The tokeziner splits the sentence into tokens and returns the IDs of each token.

In [4]:
# apply the tokenizer to the sentence and extract the token ids
token_ids = tokenizer(sentence).input_ids

In [5]:
print(token_ids)

[101, 8667, 1362, 106, 102]


To map each token ID to its corresponding token, you can use the `decode` method of the tokenizer.

In [6]:
for id in token_ids:
    print(tokenizer.decode(id))

[CLS]
Hello
world
!
[SEP]


## Visualizing Tokenization

In this section, you'll wrap the code of the previous section in the function `show_tokens`. The function takes in a text and the model name, and prints the vocabulary length of the tokenizer and a colored list of the tokens. 

In [14]:
# A list of colors in RGB for representing the tokens
colors = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence: str, tokenizer_name: str):
    """ Show the tokens each separated by a different color """

    # Load the tokenizer and tokenize the input
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids

    # Extract vocabulary length
    print(f"Vocab length: {len(tokenizer)}")

    # Print a colored list of tokens
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors[idx % len(colors)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
        )

Here's the text that you'll use to explore the different tokenization strategies of each model.

In [15]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "       "
12.0*50=600
"""

You'll now again use the tokenizer of `bert-base-cased` and compare its tokenization strategy to that of `Xenova/gpt-4`

**bert-base-cased**

In [16]:
show_tokens(text, "bert-base-cased")

Vocab length: 28996
[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " Three ta ##bs : " " 12 . 0 * 50 = 600 [SEP] 

**Optional - bert-base-uncased**

You can also try the uncased version of the bert model, and compare the vocab length and tokenization strategy of the two bert versions.

In [10]:
show_tokens(text, "bert-base-uncased")

Vocab length: 30522
[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " three tab ##s : " " 12 . 0 * 50 = 600 [SEP] 

**GPT-4**

In [11]:
show_tokens(text, "Xenova/gpt-4")

c:\Users\Nishad_Khade\OneDrive - Dell Technologies\Desktop\UI_POC\env\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Nishad_Khade\.cache\huggingface\hub\models--Xenova--gpt-4. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Vocab length: 100263

 English  and  CAPITAL IZATION 
 � � �  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         "
 12 . 0 * 50 = 600 
 

### Optional Models to Explore

You can also explore the tokenization strategy of other models. The following is a suggested list. Make sure to consider the following features when you're doing your comparison:
- Vocabulary length
- Special tokens
- Tokenization of the tabs, special characters and special keywords

**gpt2**

In [12]:
show_tokens(text, "gpt2")

Vocab length: 50257

 English  and  CAP ITAL IZ ATION 
 � � �  � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :"        "  Three  tabs :  "              " 
 12 . 0 * 50 = 600 
 